In [1]:
# import torch
# import torch_geometric as tg
import numpy as np
from pandas import read_excel
import networkx as nx
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

file_root="互联网+集成电路"


In [2]:
sup=[file for file in os.listdir(file_root) if ".xlsx" in file and "供应商" in file]
cus=[file for file in os.listdir(file_root) if ".xlsx" in file and "客户" in file]

sup_data_list=[]
sup_name_list=[]
sup_id_list=[]

cus_data_list=[]
cus_name_list=[]
cus_id_list=[]


def get_table_name(file,check):
    name,s,kind=read_excel(os.path.join(file_root,file),header=1).columns[0].split("(")
    company_id,_=s.split(")")
    kind=kind[:-1]
    if kind==check:
        table=read_excel(os.path.join(file_root,file),header=3)
        try:
            end=list(table["Ticker"]).index('来源：彭博')    # 去掉末尾项
            if len(table)<3:
                print(file)
            return name,company_id,table[:end]
        except ValueError:   # 如果没后末尾项
            return name,company_id,table
    else:
        print("wrong type")
        print(file,name,s,kind,sep="\n")
        
for file in tqdm(sup):
    sup_name,sup_id,table=get_table_name(file,check="供应商")
    sup_data_list.append(table)
    sup_id_list.append(sup_id)
    sup_name_list.append(sup_name)

for file in tqdm(cus):
    cus_name,cus_id,table=get_table_name(file,check="客户")
    cus_data_list.append(table)
    cus_id_list.append(cus_id)
    cus_name_list.append(cus_name)

  0%|          | 0/215 [00:00<?, ?it/s]

100%|██████████| 208/208 [00:06<00:00, 31.44it/s]


In [3]:
sup_data_list[0].head(1)


,Ticker,Company Name,Cost Category,%Cost Category,%Supplier Rev,Relationship Value (Q) (Mln) (USD),Relationship Year,Relationship Period,Source,As Of Date
0,2330 TT Equity,台积电,COGS,43.209843,0.300501,50.280681,2023.0,Q1,Estimate (2330 TT),2023/5/15


In [4]:
g=nx.DiGraph()


# 供应商数据
num_sup=len(sup)
for i in range(num_sup):
    company_id=sup_id_list[i] # 公司名称
    company_name=sup_name_list[i]
    
    g.add_node(company_id,name=company_name)
    company_node=g.nodes[company_id]
    
    for j,d in sup_data_list[i].iterrows():
        d=dict(d)
        sup_company_id=d.pop('Ticker')
        sup_company_name=d.pop('Company Name')
        SupRev_ratio=d.pop('%Supplier Rev')
        CateCost_ratio=d.pop('%Cost Category')
        category=d["Cost Category"]
        Q=d['Relationship Value (Q) (Mln) (USD)']
        if category not in company_node:  # 如果这类成本尚未被标记，则标记进去
            company_node[category]=Q/CateCost_ratio
        
        g.add_edge(sup_company_id,company_id,**d)  # 用字典添加边属性
        g.add_node(sup_company_id,  # 添加供应商点属性
                   **{"name":sup_company_name,
                    "Rev":Q/SupRev_ratio
                   })

# 客户数据
num_cus=len(cus)
for i in range(num_cus):
    company_id=cus_id_list[i] # 公司名称
    company_name=cus_name_list[i]
    
    g.add_node(company_id,name=company_name)
    
    for j,d in cus_data_list[i].iterrows():
        d=dict(d)
        cus_company_id=d.pop('Ticker')
        cus_company_name=d.pop('Company Name')
        Rev_ratio=d.pop('%Company Rev')
        CusCateCost_ratio=d.pop('%Customer Cost Category')
        Q=d['Relationship Value (Q) (Mln) (USD)']
        cus_category=d["Cost Category"]
                
        if not j:  # 如果是第一个就添加
            g.add_node(company_id,name=company_name,Rev=Q/Rev_ratio)
        g.add_edge(company_id,cus_company_id,**d)
        
        cus_company_node=g.nodes[cus_company_id]
        # 添加Customer Cost属性
        if cus_category not in cus_company_node:  # 如果已经有cost dict了
            cus_company_node[cus_category]=Q/CusCateCost_ratio


In [14]:
g=nx.DiGraph()
g.add_edge(1,2)
g.add_edge(1,4)
g.add_node(1,s=3)
g.add_node(1,r=5)
g.nodes[1]  # 可以连续写入信息

{'s': 3, 'r': 5}

In [33]:
g.nodes['005930 KS Equity']

{'name': '三星电子有限公司 ',
 'Rev': 519.3136198494585,
 'COGS': 316.52667558969375,
 'CAPEX': 122.61287183344908,
 'SGA': 77.97377217500055,
 nan: nan,
 nan: nan,
 nan: nan,
 nan: nan,
 nan: nan,
 'country': 'KS'}

In [22]:
g.edges[('AMD US Equity', 'META US Equity')]

{'Cost Category': 'CAPEX',
 'Relationship Value (Q) (Mln) (USD)': 109.446312,
 'Relationship Year': 2023.0,
 'Relationship Period': 'Q1',
 'Source': 'Estimate (AMD US)',
 'As Of Date': '2023/6/22'}

In [5]:
for node_id in tqdm(g.nodes):
    node=g.nodes[node_id]
    if type(node_id) != float:
        node["country"]=node_id.split(" ")[1]
    # if "Rev" not in node.keys():
    #     node["Rev"]=-1
# nx.write_graphml(g, "semi_supply_chain_more.graphml")

100%|██████████| 1381/1381 [00:00<00:00, 456340.80it/s]


In [6]:
import pandas as pd
bdp_data=pd.read_excel("点BDP更新.xlsx")
bdp_data.head(1)

,Id,name,INDUSTRY_SUBGROUP,CRNCY_ADJ_MKT_CAP,ULT_PARENT_CNTRY_OF_RISK,StockMarket,CITY_OF_DOMICILE,BS_PATENTS_TRADEMARKS_COPY_RGHTS,R&D_BDP,R&D_SUP,SGA_SUP,CAPEX_SUP,COGS_SUP,Rev_BDP,Rev_SUP,DOD_SANCTION
0,ALGM US Equity,Allegro MicroSystems股份有限公司,Semiconductor Equipment,5119.804117,JP,US,Manchester,21.971,150.85,NaN,NaN,NaN,1.16364,973.653,2.434133,U


In [7]:
# 从Excel中导入彭博的财务方面数据

g2=nx.DiGraph()
for i in g.edges:
    g2.add_edge(i[0],i[1],**g.edges[i])
    
for i,line in tqdm(bdp_data.iterrows()):
    line=dict(line)
    Id=line.pop("Id")
    g2.add_node(Id,**line)

0it [00:00, ?it/s]

1381it [00:00, 15691.68it/s]


In [9]:
g2.nodes["ALGM US Equity"]

{'name': 'Allegro MicroSystems股份有限公司',
 'INDUSTRY_SUBGROUP': 'Semiconductor Equipment',
 'CRNCY_ADJ_MKT_CAP': 5119.8041174,
 'ULT_PARENT_CNTRY_OF_RISK': 'JP',
 'StockMarket': 'US',
 'CITY_OF_DOMICILE': 'Manchester',
 'BS_PATENTS_TRADEMARKS_COPY_RGHTS': 21.971,
 'R&D_BDP': 150.85,
 'R&D_SUP': nan,
 'SGA_SUP': nan,
 'CAPEX_SUP': nan,
 'COGS_SUP': 1.16363952283752,
 'Rev_BDP': 973.653,
 'Rev_SUP': 2.4341326060606,
 'DOD_SANCTION': 'U'}

In [116]:
nx.write_gexf(g2, "semi_supply_chain_more_SeletedEdge.gexf",encoding="gbk")


In [133]:
kind_sub_edge=set()
for edge in tqdm(g2.edges):
    kind_sub_edge.add((g2.nodes[edge[0]]['INDUSTRY_SUBGROUP'],g2.nodes[edge[1]]['INDUSTRY_SUBGROUP']))
    
kind_stat={}
value_stat={}  # 统计总关系规模
for kind in kind_sub_edge:
    kind_stat[kind]=0
    value_stat[kind]=0
    
for edge in tqdm(g2.edges):
    kind=(g2.nodes[edge[0]]['INDUSTRY_SUBGROUP'],g2.nodes[edge[1]]['INDUSTRY_SUBGROUP'])
    kind_stat[kind]+=1
    
    value=g2.edges[edge]['Relationship Value (Q) (Mln) (USD)']
    if not np.isnan(value):
        value_stat[kind]+=value
    # if type(kind[0])==type(kind[1])==str:
    #     g2.edges[edge]["FromTo"]="=>".join(kind)
        
# 去掉过于稀有的类型
kind_stat2={}
value_stat2={}
for i in kind_stat:
    if kind_stat[i]>10:
        kind_stat2[i]=kind_stat[i]
        value_stat2[i]=value_stat[i]
        
for edge in tqdm(g2.edges):
    kind=(g2.nodes[edge[0]]['INDUSTRY_SUBGROUP'],g2.nodes[edge[1]]['INDUSTRY_SUBGROUP'])
    # kind_stat[kind]+=1
    if kind in kind_stat2:
        if type(kind[0])==type(kind[1])==str:  # 排除包含nan的情况
            g2.edges[edge]["FromTo"]="=>".join(kind)     


100%|██████████| 4261/4261 [00:00<00:00, 694919.10it/s]


In [85]:
pd.DataFrame([kind_stat.keys(),kind_stat.values(),value_stat.values()]).to_excel("连边类型统计.xlsx")

In [141]:
indus_stat_table=pd.read_excel("连边类型统计.xlsx")
indus_graph=nx.DiGraph()
for i,line in indus_stat_table.iterrows():
    line=dict(line)
    indus_graph.add_edge(line.pop("from"),line.pop("to"),frec=line["freq"],w=line["Q"])
indus_graph.nodes

NodeView(('Electronic Compo-Semicon', 'Semiconductor Equipment', 'Electronic Parts Distrib', 'Semicon Compo-Intg Circu', 'Computers', 'Electronic Compo-Misc', 'Audio/Video Products', 'Auto/Trk Prts&Equip-Orig', 'Telecommunication Equip', 'Auto-Cars/Light Trucks', 'Computer Services', 'Circuit Boards', 'Networking Products', 'Chemicals-Diversified', 'Retail-Consumer Electron', 'E-Commerce/Products', 'Wireless Equipment', 'Web Portals/ISP', 'Computers-Memory Devices', 'Diversified Manufact Op', 'Electronic Measur Instr', 'Cellular Telecom', 'Computers-Peripher Equip', 'Applications Software', 'Telephone-Integrated', 'Chemicals-Specialty', 'Aerospace/Defense', 'Industrial Automat/Robot', 'Distribution/Wholesale', 'Metal Processors&Fabrica', 'Electric Products-Misc', 'Machinery-Electric Util', 'Internet Content-Entmnt', 'Power Conv/Supply Equip', 'Industrial Gases', 'Lasers-Syst/Components', 'Investment Companies', 'Computers-Other', 'Advanced Materials/Prd', 'Rubber-Tires', 'Food-Retail',

In [142]:
nx.write_gml(indus_graph,"indus_graph.gml")

In [44]:
# 数据完善：
# 后续可以做的主要是：通过网络搜索获得公司的主营业务
# 对没有标签的公司进行标签推测
# 如果出度为0入度为1，这样的点基本上可以认为不是半导体相关的
# 如何知道世界上是否还有其它半导体相关的公司呢？


# 分析：
# 谁是不可或缺的一环？谁和谁是替代的关系？
# 中国半导体产业是否有可能做成自循环？
# 


# 一些初步洞见：
# 

AMD US Equity


In [14]:
nx.write_graphml(g2,"python_graph.graphml")

In [15]:
g3=nx.read_graphml("python_graph.graphml")